In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import metapack as mp
import numpy as np
mp.jupyter.init() # Makes 'index:' url work
import metapack as mp

#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg.set_sys_path()
import pylib


In [2]:
from IPython.display import display, clear_output
fp = pkg.reference('lines').resolved_url.fspath


/Users/eric/proj/code-projects/radius-search/staging/civicknowledge.com-osm-demosearch/data/csv/lines.csv


In [3]:
cache = pylib.open_cache(pkg)

print('-- Split the input file')
%time splits = pylib.split_data(pkg, cache)
print('-- Run the overlay process')
%time recombine_keys = pylib.run_overlay(splits, cache)

print('-- Simplify lines')
%time simplified_keys = pylib.simplify_lines(cache, recombine_keys)

print('--- Write the roads files')
pylib.write_files(pkg, simplified_keys)


-- Split the input file
CPU times: user 691 µs, sys: 204 µs, total: 895 µs
Wall time: 898 µs
-- Run the overlay process
CPU times: user 736 µs, sys: 251 µs, total: 987 µs
Wall time: 992 µs
